<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"></ul></div>

# Importar os módulos

In [31]:
%load_ext autoreload
%autoreload 2
import os
import pandas as pd
import numpy as np
import datetime
import seaborn as sns
import matplotlib.pyplot as plt
import Dataset_transf as dprep
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Obter os dataframes dos ficheiros

In [2]:
failures_df, signals_df, logs_df = dprep.get_data()

In [3]:
df_list = [failures_df, signals_df, logs_df]

# Criação de datasets de components

In [4]:
df_generator, df_gen_bear, df_transformer, df_hydraulic, df_gearbox = dprep.component_df_creation(signals_df)

In [5]:
comp_df_list = [df_generator, df_hydraulic, df_gen_bear, df_transformer,  df_gearbox]

In [6]:
for i in comp_df_list:
    print(i.shape)

(434145, 45)
(434145, 31)
(434145, 45)
(434145, 36)
(434145, 33)


In [7]:
component_list = failures_df.Component.unique()
component_list

array(['GENERATOR', 'HYDRAULIC_GROUP', 'GENERATOR_BEARING', 'TRANSFORMER',
       'GEARBOX'], dtype=object)

# Merge dos DF com o DF de failures

In [8]:
for i in range(len(component_list)):
    comp_df_list[i] = dprep.sig_fail_merge_dfs(sig_df=comp_df_list[i],
                                               fail_df=failures_df,
                                               component=component_list[i])

In [9]:
for i in comp_df_list:
    print(i.shape)

(434147, 46)
(434145, 32)
(434145, 46)
(434145, 37)
(434145, 34)


# Fillna by turbine

In [10]:
turbine_list = signals_df.Turbine_ID.unique()
turbine_list

array(['T11', 'T06', 'T01', 'T09', 'T07'], dtype=object)

In [11]:
for i in range(len(component_list)):
    comp_df_list[i] = dprep.fill_na_by_turbine(comp_df_list[i], turbine_list)

C:\Users\ricar\code\RicardoJPedro82\wmillfailprev\notebooks\Dataset_transf.py:149: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  index['date'] = index['Timestamp']
C:\Users\ricar\code\RicardoJPedro82\wmillfailprev\notebooks\Dataset_transf.py:170: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  df_ = pd.concat([df_, df_final])
C:\Users\ricar\code\RicardoJPedro82\wmillfailprev\notebooks\Dataset_transf.py:162: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead


In [12]:
for i in comp_df_list:
    print(i.shape)

(434147, 47)
(434145, 33)
(434145, 47)
(434145, 38)
(434145, 35)


# Criação de Variável alvo

In [13]:
for i in range(len(comp_df_list)):
    print(comp_df_list[i].isna().sum().value_counts())

0    47
dtype: int64
0    33
dtype: int64
0    47
dtype: int64
0    38
dtype: int64
0    35
dtype: int64


In [14]:
days_list = [60, 45, 40, 30, 10, 5]

In [15]:
for day in days_list:
    for i in range(len(comp_df_list)):
        comp_df_list[i] = dprep.aplic_var_target(comp_df_list[i], day)

In [16]:
for i in comp_df_list:
    print(i.shape)

(434147, 50)
(434145, 36)
(434145, 50)
(434145, 41)
(434145, 38)


# Feature selection

In [17]:
for i in comp_df_list:
    print(i.keys())

Index(['Amb_Temp_Avg', 'Amb_WindDir_Abs_Avg', 'Amb_WindDir_Relative_Avg',
       'Amb_WindSpeed_Avg', 'Amb_WindSpeed_Est_Avg', 'Amb_WindSpeed_Max',
       'Amb_WindSpeed_Min', 'Amb_WindSpeed_Std', 'Blds_PitchAngle_Avg',
       'Blds_PitchAngle_Max', 'Blds_PitchAngle_Min', 'Blds_PitchAngle_Std',
       'Component', 'Cont_Hub_Temp_Avg', 'Cont_Top_Temp_Avg',
       'Cont_VCP_ChokcoilTemp_Avg', 'Cont_VCP_Temp_Avg',
       'Cont_VCP_WtrTemp_Avg', 'Gen_Bear2_Temp_Avg', 'Gen_Bear_Temp_Avg',
       'Gen_Phase1_Temp_Avg', 'Gen_Phase2_Temp_Avg', 'Gen_Phase3_Temp_Avg',
       'Gen_RPM_Avg', 'Gen_RPM_Max', 'Gen_RPM_Min', 'Gen_RPM_Std',
       'Gen_SlipRing_Temp_Avg', 'Grd_Busbar_Temp_Avg',
       'Grd_RtrInvPhase1_Temp_Avg', 'Grd_RtrInvPhase2_Temp_Avg',
       'Grd_RtrInvPhase3_Temp_Avg', 'Hyd_Oil_Temp_Avg', 'Nac_Direction_Avg',
       'Nac_Temp_Avg', 'Prod_LatestAvg_ActPwrGen0',
       'Prod_LatestAvg_ActPwrGen1', 'Prod_LatestAvg_ReactPwrGen0',
       'Prod_LatestAvg_ReactPwrGen1', 'Rtr_RPM_Avg',

In [18]:
gen_features_drop = [
    'Gen_RPM_Max', 'Gen_RPM_Min', 'Gen_Phase1_Temp_Avg', 'Gen_Phase3_Temp_Avg',
    'Amb_WindSpeed_Est_Avg', 'Grd_RtrInvPhase1_Temp_Avg',
    'Grd_RtrInvPhase3_Temp_Avg', 'Rtr_RPM_Max', 'Rtr_RPM_Min',
    'Blds_PitchAngle_Max', 'Blds_PitchAngle_Min',
    'Prod_LatestAvg_ReactPwrGen1', 'Cont_Hub_Temp_Avg', 'Spin_Temp_Avg',
    'Rtr_RPM_Std', 'Rtr_RPM_Avg', 'Cont_VCP_Temp_Avg'
]

In [19]:
gen_bear_features_drop = [
    'Gen_RPM_Max', 'Gen_RPM_Min', 'Gen_Phase1_Temp_Avg', 'Gen_Phase3_Temp_Avg',
    'Amb_WindSpeed_Est_Avg', 'Grd_RtrInvPhase1_Temp_Avg',
    'Grd_RtrInvPhase3_Temp_Avg', 'Rtr_RPM_Max', 'Rtr_RPM_Min',
    'Blds_PitchAngle_Max', 'Blds_PitchAngle_Min',
    'Prod_LatestAvg_ReactPwrGen1', 'Cont_Hub_Temp_Avg', 'Spin_Temp_Avg',
    'Rtr_RPM_Std', 'Rtr_RPM_Avg', 'Cont_VCP_Temp_Avg'
]

In [20]:
hyd_features_drop = [
    'Rtr_RPM_Max', 'Rtr_RPM_Min', 'Blds_PitchAngle_Max', 'Blds_PitchAngle_Min',
    'Blds_PitchAngle_Max', 'Grd_RtrInvPhase3_Temp_Avg', 'Grd_Busbar_Temp_Avg',
    'Amb_WindSpeed_Est_Avg', 'Spin_Temp_Avg', 'Cont_Hub_Temp_Avg',
    'Grd_RtrInvPhase1_Temp_Avg', 'Cont_VCP_Temp_Avg'
]

In [21]:
gearbox_features_drop = [
    'Rtr_RPM_Max', 'Rtr_RPM_Min', 'Grd_RtrInvPhase1_Temp_Avg',
    'Grd_RtrInvPhase3_Temp_Avg', 'Blds_PitchAngle_Min', 'Blds_PitchAngle_Max',
    'Cont_VCP_Temp_Avg', 'Grd_Busbar_Temp_Avg', 'Amb_WindSpeed_Est_Avg',
    'Spin_Temp_Avg'
]

In [22]:
transf_features_drop = [
    'HVTrafo_Phase1_Temp_Avg', 'HVTrafo_Phase3_Temp_Avg', 'Rtr_RPM_Max',
    'Rtr_RPM_Min', 'Grd_RtrInvPhase1_Temp_Avg', 'Grd_RtrInvPhase3_Temp_Avg',
    'Blds_PitchAngle_Min', 'Blds_PitchAngle_Max', 'Amb_WindSpeed_Est_Avg',
    'Spin_Temp_Avg', 'Cont_VCP_Temp_Avg'
]

In [23]:
feat_drop_list = [
    gen_features_drop, hyd_features_drop, gen_bear_features_drop,
    transf_features_drop, gearbox_features_drop
]

In [28]:
for df in range(len(comp_df_list)):
    print(f'A actualizar dataframe {df+1} de {len(comp_df_list)}')
#     for col in feat_drop_list:
#         print(f'A apagar a coluna {col}')
    comp_df_list[df] = comp_df_list[df].drop(columns=feat_drop_list[df])

A actualizar dataframe 1 de 5
A actualizar dataframe 2 de 5
A actualizar dataframe 3 de 5
A actualizar dataframe 4 de 5
A actualizar dataframe 5 de 5


In [29]:
for i in comp_df_list:
    print(i.shape)

(434147, 33)
(434145, 25)
(434145, 33)
(434145, 30)
(434145, 28)


In [30]:
for i in comp_df_list:
    print(i.keys())

Index(['Amb_Temp_Avg', 'Amb_WindDir_Abs_Avg', 'Amb_WindDir_Relative_Avg',
       'Amb_WindSpeed_Avg', 'Amb_WindSpeed_Max', 'Amb_WindSpeed_Min',
       'Amb_WindSpeed_Std', 'Blds_PitchAngle_Avg', 'Blds_PitchAngle_Std',
       'Component', 'Cont_Top_Temp_Avg', 'Cont_VCP_ChokcoilTemp_Avg',
       'Cont_VCP_WtrTemp_Avg', 'Gen_Bear2_Temp_Avg', 'Gen_Bear_Temp_Avg',
       'Gen_Phase2_Temp_Avg', 'Gen_RPM_Avg', 'Gen_RPM_Std',
       'Gen_SlipRing_Temp_Avg', 'Grd_Busbar_Temp_Avg',
       'Grd_RtrInvPhase2_Temp_Avg', 'Hyd_Oil_Temp_Avg', 'Nac_Direction_Avg',
       'Nac_Temp_Avg', 'Prod_LatestAvg_ActPwrGen0',
       'Prod_LatestAvg_ActPwrGen1', 'Prod_LatestAvg_ReactPwrGen0', 'TTF',
       'Timestamp', 'Turbine_ID', '60_days', '30_days', '10_days'],
      dtype='object')
Index(['Amb_Temp_Avg', 'Amb_WindDir_Abs_Avg', 'Amb_WindDir_Relative_Avg',
       'Amb_WindSpeed_Avg', 'Amb_WindSpeed_Max', 'Amb_WindSpeed_Min',
       'Amb_WindSpeed_Std', 'Blds_PitchAngle_Avg', 'Blds_PitchAngle_Std',
       'Comp

# Train and test split

In [45]:
# Preparar a lista de X_train
df_train = [0, 0, 0, 0, 0]
df_test = [0, 0, 0, 0, 0]

for i in range(len(comp_df_list)):
    df_train[i] = dprep.prepare_train_df(comp_df_list[i])
    df_test[i] = dprep.prepare_test_df(comp_df_list[i])

In [46]:
for i in range(len(df_train)):
    print(f'x_train {i} com a forma {df_train[i].shape}')
    print(f'x_test {i} com a forma {df_test[i].shape}')

x_train 0 com a forma (368857, 33)
x_test 0 com a forma (65290, 33)
x_train 1 com a forma (368855, 25)
x_test 1 com a forma (65290, 25)
x_train 2 com a forma (368855, 33)
x_test 2 com a forma (65290, 33)
x_train 3 com a forma (368855, 30)
x_test 3 com a forma (65290, 30)
x_train 4 com a forma (368855, 28)
x_test 4 com a forma (65290, 28)


In [47]:
df_train[0]

,Amb_Temp_Avg,Amb_WindDir_Abs_Avg,Amb_WindDir_Relative_Avg,Amb_WindSpeed_Avg,Amb_WindSpeed_Max,Amb_WindSpeed_Min,Amb_WindSpeed_Std,Blds_PitchAngle_Avg,Blds_PitchAngle_Std,Component,...,Nac_Temp_Avg,Prod_LatestAvg_ActPwrGen0,Prod_LatestAvg_ActPwrGen1,Prod_LatestAvg_ReactPwrGen0,TTF,Timestamp,Turbine_ID,60_days,30_days,10_days
0,18.0,199.1,-7.8,5.3,8.6,2.6,0.7,-1.4,0.3,0.0,...,27.0,0.0,41059.0,0.0,63.0,2016-01-01 00:00:00,T11,0,0,0
1,18.0,207.5,0.6,5.7,11.1,1.7,0.9,-1.7,0.4,0.0,...,27.0,0.0,50808.0,0.0,63.0,2016-01-01 00:10:00,T11,0,0,0
2,18.0,190.5,-16.5,6.1,9.9,1.5,0.8,-1.9,0.2,0.0,...,27.0,0.0,60667.0,0.0,63.0,2016-01-01 00:20:00,T11,0,0,0
3,18.0,214.6,7.6,6.3,9.4,2.7,0.8,-2.0,0.2,0.0,...,27.0,0.0,77486.0,0.0,63.0,2016-01-01 00:30:00,T11,0,0,0
4,18.0,212.1,5.2,6.4,11.6,1.7,0.9,-2.0,0.3,0.0,...,27.0,0.0,67378.0,0.0,63.0,2016-01-01 00:40:00,T11,0,0,0
5,18.0,212.1,5.2,6.2,21.7,1.9,0.8,-2.0,0.3,0.0,...,27.0,0.0,63730.0,0.0,63.0,2016-01-01 00:50:00,T11,0,0,0
6,18.0,218.4,11.5,5.5,8.5,2.7,0.8,-1.5,0.4,0.0,...,27.0,0.0,42351.0,0.0,63.0,2016-01-01 01:00:00,T11,0,0,0
7,18.0,194.9,-12.0,5.0,7.7,1.6,0.7,-1.1,0.3,0.0,...,27.0,0.0,34552.0,0.0,63.0,2016-01-01 01:10:00,T11,0,0,0
8,18.0,219.5,12.6,5.2,8.8,1.7,0.8,-1.3,0.5,0.0,...,27.0,0.0,35078.0,0.0,63.0,2016-01-01 01:20:00,T11,0,0,0
9,18.0,194.4,-21.2,4.9,7.3,2.1,0.7,-1.0,0.3,0.0,...,27.0,0.0,28542.0,0.0,63.0,2016-01-01 01:30:00,T11,0,0,0
